In [1]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install open3d

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pythreejs

     ---------------------------------------- 3.4/3.4 MB 2.9 MB/s eta 0:00:00
     -------------------------------------- 271.7/271.7 kB 1.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install ipyvolume

     ---------------------------------------- 1.6/1.6 MB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 11.7/11.7 MB 2.2 MB/s eta 0:00:00
     -------------------------------------- 260.7/260.7 kB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 2.8 MB/s eta 0:00:00
     ---------------------------------------- 2.7/2.7 MB 3.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install trimesh

     -------------------------------------- 681.6/681.6 kB 1.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install flask

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pyglet


     -------------------------------------- 853.1/853.1 kB 1.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install pyvista

     ---------------------------------------- 1.6/1.6 MB 2.0 MB/s eta 0:00:00
     ---------------------------------------- 60.9/60.9 kB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 48.8/48.8 MB 1.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install "pyglet<2"


     ---------------------------------------- 1.1/1.1 MB 529.4 kB/s eta 0:00:00
  Attempting uninstall: pyglet
    Found existing installation: pyglet 2.0.8
    Uninstalling pyglet-2.0.8:
      Successfully uninstalled pyglet-2.0.8
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install trimesh

Note: you may need to restart the kernel to use updated packages.


In [1]:
import gradio as gr
import numpy as np
import open3d as o3d
import os
from PIL import Image
import tempfile
import torch
from transformers import GLPNImageProcessor, GLPNForDepthEstimation
import trimesh
import pyvista as pv

feature_extractor = GLPNImageProcessor.from_pretrained("vinvino02/glpn-nyu")
model = GLPNForDepthEstimation.from_pretrained("vinvino02/glpn-nyu")

def process_image(image, quality):
    # Predict depth using GLPN model
    new_height = 480 if image.height > 480 else image.height
    new_height -= (new_height % 32)
    new_width = int(new_height * image.width / image.height)
    diff = new_width % 32
    new_width = new_width - diff if diff < 16 else new_width + 32 - diff
    new_size = (new_width, new_height)
    image = image.resize(new_size)
    inputs = feature_extractor(images=image, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        predicted_depth = outputs.predicted_depth
    output = predicted_depth.squeeze().cpu().numpy() * 1000.0
    pad = 16
    output = output[pad:-pad, pad:-pad]
    image = image.crop((pad, pad, image.width - pad, image.height - pad))

    # Generate mesh using Open3D
    width, height = image.size
    depth_image = (output * 255 / np.max(output)).astype('uint8')
    image_np = np.array(image)
    depth_o3d = o3d.geometry.Image(depth_image)
    image_o3d = o3d.geometry.Image(image_np)
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(image_o3d, depth_o3d, convert_rgb_to_intensity=False)
    camera_intrinsic = o3d.camera.PinholeCameraIntrinsic()
    camera_intrinsic.set_intrinsics(width, height, 500, 500, width / 2, height / 2)
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, camera_intrinsic)
    cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=20.0)
    pcd = pcd.select_by_index(ind)
    pcd.estimate_normals()
    pcd.orient_normals_to_align_with_direction(orientation_reference=(0., 0., -1.))
    mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=quality + 5, n_threads=1)[0]
    rotation = mesh.get_rotation_matrix_from_xyz((np.pi, np.pi, 0))
    mesh.rotate(rotation, center=(0, 0, 0))
    temp_name = next(tempfile._get_candidate_names()) + '.obj'
    o3d.io.write_triangle_mesh(temp_name, mesh)

    return depth_image, temp_name

def generate_mesh_preview(mesh_path):
    mesh = trimesh.load_mesh(mesh_path)
    pv_mesh = pv.wrap(mesh)
    pv_mesh.plot()

def process_image_with_gradio(image, quality):
    depth_image, mesh_path = process_image(image, quality)
    generate_mesh_preview(mesh_path)
    return depth_image, mesh_path

iface = gr.Interface(
    fn=process_image_with_gradio,
    inputs=[
        gr.inputs.Image(type='pil', label='Drag and Drop or Upload an Image'),
        gr.inputs.Slider(1, 5, step=1, default=3, label='Mesh Quality')
    ],
    outputs=[
        gr.outputs.Image(type='numpy', label='Depth Map'),
        gr.outputs.File(label='Download Mesh (.obj)'),
    ],
    title='T.O.R.A',
    description='''<div style="text-align:center;">Transforming Object Reconstructing Application</div>''',
    allow_flagging=False
)

iface.launch(inline=True)



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


C:\Users\praka\anaconda3\lib\site-packages\gradio\inputs.py:259: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
C:\Users\praka\anaconda3\lib\site-packages\gradio\inputs.py:262: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
C:\Users\praka\anaconda3\lib\site-packages\gradio\inputs.py:89: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
C:\Users\praka\anaconda3\lib\site-packages\gradio\inputs.py:93: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
C:\Users\praka\anaconda3\lib\site-packages\gradio\outputs.py:43: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
C:\Users\pra

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
